In [2]:
import nltk
import pandas as pd
import re
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof
import os
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [3]:
# Mendapatkan daftar file dalam folder
folder_path = 'tweets-data'  # Ganti dengan jalur folder yang sesuai
files = os.listdir(folder_path)

# Mencari file CSV terahir berdasarkan data modifikasi
latest_file = None
latest_modified_time = 0

for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        modified_time = os.path.getmtime(file_path)
        
        if modified_time > latest_modified_time:
            latest_modified_time = modified_time
            latest_file = file_path

# Membaca file CSV terakhir menggunakan pandas
if latest_file is not None:
    df = pd.read_csv(latest_file, delimiter=";")
    print(df.head())
else:
    print("Tidak ada file CSV dalam folder tersebut.")


                       created_at                   id               id_str  \
0  Mon May 29 14:41:33 +0000 2023  1663193866219016200  1663193866219016195   
1  Mon May 29 15:28:23 +0000 2023  1663205656009248800  1663205656009248769   
2  Mon May 29 14:23:58 +0000 2023  1663189443640189000  1663189443640188928   
3  Mon May 29 23:23:35 +0000 2023  1663325242897600500  1663325242897600512   
4  Mon May 29 23:20:34 +0000 2023  1663324481807601700  1663324481807601666   

                                           full_text  quote_count  \
0  @SidVanz @Box2BoxBola Mmmm, town ~ kota kecil,...            0   
1                  Di kota tegal namanya gula kacang            0   
2  GBB dan FSB Garteks KSBSI se-Jawa Tengah mengg...            0   
3  06.15 Situasi arus lalu-lintas di Tol Dalam Ko...            0   
4  Menerima Anak PKL, 0812-2134-9398 Info Tempat ...            0   

   reply_count  retweet_count  favorite_count  geo lang          user_id_str  \
0            0              0 

In [4]:
new_df=df[["created_at", "full_text"]]
new_df['created_at'] = pd.to_datetime(new_df['created_at'], format="%a %b %d %H:%M:%S %z %Y")

# Extract the date portion in yyyy-mm-dd format
new_df['formatted_date'] = new_df['created_at'].dt.strftime("%Y-%m-%d")
del new_df['created_at']
new_df = new_df[['formatted_date'] + list(new_df.columns[:-1])]

C:\Users\iki11\AppData\Local\Temp\ipykernel_6672\2513206295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['created_at'] = pd.to_datetime(new_df['created_at'], format="%a %b %d %H:%M:%S %z %Y")
C:\Users\iki11\AppData\Local\Temp\ipykernel_6672\2513206295.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['formatted_date'] = new_df['created_at'].dt.strftime("%Y-%m-%d")


In [6]:
df = new_df.rename(columns={'formatted_date': 'tanggal', 'full_text': 'tweet'})
df.head()

,tanggal,tweet
0,2023-05-29,"@SidVanz @Box2BoxBola Mmmm, town ~ kota kecil,..."
1,2023-05-29,Di kota tegal namanya gula kacang
2,2023-05-29,GBB dan FSB Garteks KSBSI se-Jawa Tengah mengg...
3,2023-05-29,06.15 Situasi arus lalu-lintas di Tol Dalam Ko...
4,2023-05-29,"Menerima Anak PKL, 0812-2134-9398 Info Tempat ..."


In [89]:
#menyimpan tweet. (tipe data series pandas)
data_content = df['tweet']

In [90]:
# casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    @sidvanz @box2boxbola mmmm, town ~ kota kecil,...
1                    di kota tegal namanya gula kacang
2    gbb dan fsb garteks ksbsi se-jawa tengah mengg...
3    06.15 situasi arus lalu-lintas di tol dalam ko...
4    buka akun xm ultra low hanya dengan $5 🥳\n\n👉 ...
Name: tweet, dtype: object

In [94]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", str(tweet)) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


In [95]:
data_filtering

0        sidvanz  box boxbola mmmm  town   kota kecil ...
1                       di kota tegal namanya gula kacang
2       gbb dan fsb garteks ksbsi se jawa tengah mengg...
3           situasi arus lalu lintas di tol dalam kota...
4       buka akun xm ultra low hanya dengan    🥳\n\n👉 ...
                              ...                        
2307    hai  sobatjdih \ndiinformasikan bahwa pada har...
2308    baru ngeh akun ig pemkot tegal pake foto profi...
2309    hai  sobatjdih \ndiinformasikan bahwa pada har...
2310    suwarsih  pensiunan guru sd jadi calon jemaah ...
2311    hai  sobatjdih \ndiinformasikan bahwa pada har...
Length: 2312, dtype: object

In [96]:
# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]
data_tokenize

[['sidvanz',
  'box',
  'boxbola',
  'mmmm',
  'town',
  'kota',
  'kecil',
  'city',
  'kota',
  'besar',
  '\\',
  'nagak',
  'beda',
  'aturannya',
  'dengan',
  'indonesia',
  'mungkin',
  'kalau',
  'di',
  'indonesia',
  'seperti',
  'kota',
  'tegal'],
 ['di', 'kota', 'tegal', 'namanya', 'gula', 'kacang'],
 ['gbb',
  'dan',
  'fsb',
  'garteks',
  'ksbsi',
  'se',
  'jawa',
  'tengah',
  'menggelar',
  'rapat',
  'konsolidasi',
  'u',
  'pemenangan',
  'ganjarpranowo',
  'pd',
  'pilpres',
  'di',
  'semarang',
  'jumat',
  'dihadiri',
  'an',
  'orang',
  'yg',
  'mewakili',
  'dpc',
  'fsb',
  'garteks',
  'se',
  'jateng',
  'kota',
  'semarang',
  'kab',
  'semarang',
  'solo',
  'raya',
  'tegal',
  'raya',
  'pekalongan',
  'jepara'],
 ['situasi',
  'arus',
  'lalu',
  'lintas',
  'di',
  'tol',
  'dalam',
  'kota',
  'exit',
  'tegal',
  'parang',
  'dan',
  'arteri',
  'jl',
  'gatot',
  'subroto',
  'dari',
  'cawang',
  'menuju',
  'semanggi',
  'terpantau',
  'ramai',

In [97]:
#slang word
path_dataslang = open("kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

In [98]:
dataslang

,0,1
0,tlg,tolong
1,dwpan,depan
2,dwngan,dengan
3,byk,banyak
4,smoga,semoga
...,...,...
5437,politic,politik
5438,cont,
5439,rt,
5440,cc,NaN


In [99]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
reviews = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
reviews

[['sidvanz',
  'box',
  'boxbola',
  'mmmm',
  'town',
  'kota',
  'city',
  'kota',
  '\\',
  'nagak',
  'beda',
  'aturannya',
  'indonesia',
  'indonesia',
  'kota',
  'tegal'],
 ['kota', 'tegal', 'namanya', 'gula', 'kacang'],
 ['gbb',
  'fsb',
  'garteks',
  'ksbsi',
  'jawa',
  'menggelar',
  'rapat',
  'konsolidasi',
  'pemenangan',
  'ganjarpranowo',
  'pilpres',
  'semarang',
  'jumat',
  'dihadiri',
  'an',
  'orang',
  'mewakili',
  'dpc',
  'fsb',
  'garteks',
  'jateng',
  'kota',
  'semarang',
  'kab',
  'semarang',
  'solo',
  'raya',
  'tegal',
  'raya',
  'pekalongan',
  'jepara'],
 ['situasi',
  'arus',
  'lintas',
  'tol',
  'kota',
  'exit',
  'tegal',
  'parang',
  'arteri',
  'jl',
  'gatot',
  'subroto',
  'cawang',
  'semanggi',
  'terpantau',
  'ramai',
  'lancar',
  'pagi'],
 ['buka',
  'akun',
  'xm',
  'ultra',
  'low',
  '🥳',
  '\\',
  '\\',
  '👉',
  'spread',
  '\\',
  '👉',
  'leverage',
  '\\',
  '👉',
  'pilih',
  'instrumen',
  '\\',
  '👉',
  'opsi',
  'a

In [109]:
# Specify the file path of the pickle file
file_path = 'model2\data_train.pickle'

# Read the pickle file
with open(file_path, 'rb') as file:
    data_train = pickle.load(file)

In [111]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
train_vector = vectorizer.fit_transform(data_train)
reviews2 = [" ".join(r) for r in reviews]

In [107]:
reviews2

['sidvanz box boxbola mmmm town kota city kota \\ nagak beda aturannya indonesia indonesia kota tegal',
 'kota tegal namanya gula kacang',
 'gbb fsb garteks ksbsi jawa menggelar rapat konsolidasi pemenangan ganjarpranowo pilpres semarang jumat dihadiri an orang mewakili dpc fsb garteks jateng kota semarang kab semarang solo raya tegal raya pekalongan jepara',
 'situasi arus lintas tol kota exit tegal parang arteri jl gatot subroto cawang semanggi terpantau ramai lancar pagi',
 'buka akun xm ultra low 🥳 \\ \\ 👉 spread \\ 👉 leverage \\ 👉 pilih instrumen \\ 👉 opsi akun islami tersedia \\ \\ xm lowspreads ultralowaccount',
 'menerima anak pkl info prakerin menerima pogram pkl smk yogyakarta kota tegal',
 'minggu \\ nnuri kriswanti si luhkan kota magelang minggu produktif pokdakan jambon tegal lele semangat menyelesaikan kolam budidaya cbib luhlapor giatluhkansatminkaltegal pkblu bp tegal puslatluhkp brsdm kp',
 'tmcpoldametro situasi arus lintas tol kota exit tegal parang arteri jl gatot s

## Hiburan

In [126]:
model_aspek = pickle.load(open('model2/tfidf_Model_Aspek_Wisata_Hiburan_nvb.pkl','rb'))
model_sentiment = pickle.load(open('model2/vektor_tfidf_Model_Sentimen_Wisata_Hiburan_nvb.pkl','rb'))

result = []

for test in reviews2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data)
    pred = model_aspek.predict(test_vector)
    if pred != 1:
        result.append(-1)
    else:
        pred = model_sentiment.predict(test_vector)
        result.append(pred[0])

        
from sklearn.utils.multiclass import unique_labels
unique_labels(result)

df['wisata_hiburan'] = result

## Pendidikan

In [127]:
model_aspek = pickle.load(open('model2/tfidf_Model_Aspek_Wisata_Pendidikan_nvb.pkl','rb'))
model_sentiment = pickle.load(open('model2/vektor_tfidf_Model_Sentimen_Pendidikan_nvb.pkl','rb'))

result = []

for test in reviews2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data)
    pred = model_aspek.predict(test_vector)
    if pred != 1:
        result.append(-1)
    else:
        pred = model_sentiment.predict(test_vector)
        result.append(pred[0])

from sklearn.utils.multiclass import unique_labels
unique_labels(result)

df['pendidikan'] = result

## Fasilitas

In [128]:
model_aspek = pickle.load(open('model2/tfidf_Model_Aspek_Wisata_Fasilitas_Layanan_Publik_nvb.pkl','rb'))
model_sentiment = pickle.load(open('model2/vektor_tfidf_Model_Sentimen_fasilitas_layanan_publik_nvb.pkl','rb'))

result = []

for test in reviews2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data)
    pred = model_aspek.predict(test_vector)
    if pred != 1:
        result.append(-1)
    else:
        pred = model_sentiment.predict(test_vector)
        result.append(pred[0])

from sklearn.utils.multiclass import unique_labels
unique_labels(result)

df['fasilitas_layanan_publik'] = result

## Kuliner

In [129]:
model_aspek = pickle.load(open('model2/tfidf_Model_Aspek_Wisata_Kuliner_nvb.pkl','rb'))
model_sentiment = pickle.load(open('model2/vektor_tfidf_Model_Sentimen_Kuliner_nvb.pkl','rb'))

result = []

for test in reviews2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data)
    pred = model_aspek.predict(test_vector)
    if pred != 1:
        result.append(-1)
    else:
        pred = model_sentiment.predict(test_vector)
        result.append(pred[0])

from sklearn.utils.multiclass import unique_labels
unique_labels(result)

df['kuliner'] = result

In [135]:
df=df.dropna()
df=df.drop_duplicates()
df.head()

,tanggal,tweet,wisata_hiburan,pendidikan,fasilitas_layanan_publik,kuliner
0,2023-05-29,"@SidVanz @Box2BoxBola Mmmm, town ~ kota kecil,...",-1,-1,-1,-1
1,2023-05-29,Di kota tegal namanya gula kacang,-1,-1,-1,1
2,2023-05-29,GBB dan FSB Garteks KSBSI se-Jawa Tengah mengg...,-1,-1,-1,-1
3,2023-05-29,06.15 Situasi arus lalu-lintas di Tol Dalam Ko...,-1,-1,-1,-1
4,2023-05-05,Buka akun XM Ultra Low hanya dengan $5 🥳\n\n👉 ...,-1,-1,-1,1


In [136]:
import pymysql
def insert_to_list_rekomen_db(rp2):
    # Connect to the database
    connection = pymysql.connect(host='127.0.0.1',
                                 port=3306,
                                 user='root',
                                 # password='f#Ur8J3N',
                                 database='dashboard_kotategal_db')
    # create cursor
    cursor=connection.cursor()
    
    cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
    for i,row in rp2.iterrows():
        sql = "INSERT INTO `hasil_sentiment` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()
insert_to_list_rekomen_db(rp2=df)